## Import essential packages

In [1]:
#----- These are for the simulation 

from __future__ import division
import hoomd
import hoomd.md
import numpy
from matplotlib import pyplot

#----- These are for the orientation analysis

import gsd
import gsd.hoomd

# Simulation

In [2]:
#-----Some important variables

box = 12.5
replicates = 10
p_max = 0.25
t_max = 1.0

In [3]:
#-------Define the unit cell

sim = hoomd.context.initialize("");
uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----From the lattice we obtain a snapshot and initialize.

snapshot = uc.get_snapshot()
snapshot.replicate(replicates,replicates,replicates)
system = hoomd.init.read_snapshot(snapshot)

#-----Add the particle type for the constituent particles.

system.particles.types.add('A');

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)


#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

pressure = hoomd.variant.linear_interp(points = [(0,1e-6), (1e4, p_max)])

     #-----Temperatures

t1 = hoomd.variant.linear_interp(points = [(0,0.5), (1e4, 1.0)])
t2 = hoomd.variant.linear_interp(points = [(0,1.0), (1e4, 1.2)])
t3 = hoomd.variant.linear_interp(points = [(0,1.2), (1e4, 1.4)])
t4 = hoomd.variant.linear_interp(points = [(0,1.4), (1e4, 1.6)])
t5 = hoomd.variant.linear_interp(points = [(0,1.6), (1e4, 1.8)])
t6 = hoomd.variant.linear_interp(points = [(0,1.8), (1e4, 2.0)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t1, tau = 0.5, tauP = 1.0, P = pressure);
npt.randomize_velocities(seed=42)


#-----Write output


gsd_file = "global_trayectory.gsd"

gsd = hoomd.dump.gsd(gsd_file,
               period=1e3,
               group=hoomd.group.all(),
               overwrite=True);  


log_file = "global_trayectory.log"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period=1e3,
                         overwrite=True);

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


## Run the simulation

In [4]:
with sim:
    hoomd.run(6e4)
    npt.set_params(kT = t2)
    hoomd.run(6e4)
    npt.set_params(kT = t3)
    hoomd.run(6e4)
    npt.set_params(kT = t4)
    hoomd.run(6e4)
    npt.set_params(kT = t5)
    hoomd.run(6e4)
    npt.set_params(kT = t6)
    hoomd.run(6e4)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 618 / 60000 | TPS 61.7275 | ETA 00:16:02
Time 00:00:20 | Step 866 / 60000 | TPS 24.7531 | ETA 00:39:48
Time 00:00:30 | Step 1109 / 60000 | TPS 24.0463 | ETA 00:40:49
Time 00:00:40 | Step 1321 / 60000 | TPS 21.1713 | ETA 00:46:11
Time 00:00:50 | Step 1535 / 60000 | TPS 21.1733 | ETA 00:46:01
Time 00:01:00 | Step 1755 / 60000 | TPS 21.8732 | ETA 00:44:22
Time 00:01:10 | Step 1950 / 60000 | TPS 19.4701 | ETA 00:49:41
Time 00:01:20 | Step 2157 / 60000 | TPS 20.2914 | ETA 00:47:30
Time 00:01:30 | Step 2344 / 60000 | TPS 18.3475 | ETA 00:52:22
Time 00:01:40 | Step 2533 / 60000 | TPS 18.8516 | ETA 00:50:48
Time 00:01:50 | Step 2700 / 60000 | TPS 16.6811 | ETA 00:57:15
Time 00:02:00 | Step 2860 / 60000 | TPS 15.996 | ETA 00:59:32
Time

**ERROR**: Particle with unique tag 8204 has NaN for its position.


RuntimeError: Error computing cell list

## System initial density

In [ ]:
particles = snapshot.particles.N

lx = snapshot.box.Lx
ly = snapshot.box.Ly
lz = snapshot.box.Lz
volume = lx * ly *lz

rho = particles / volume

print("Density = ", rho)

In [ ]:
system.get_metadata()